<a href="https://colab.research.google.com/github/Doryane/Big_data/blob/main/Web_Scrapping_AO3_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import time
start_time = time.time()
#main()

Web Scripping pour AO3 based on following code :
https://github.com/kenalba/ao3-scraper/blob/main/scraper.py



---

# What this code do ?

With this code you can append the following information from AO3 and put it into a data frame from a certain page to another

*  Story IDS of the work
*  Updated date of the work
*  Title and author of the work
*  Language of the work
*  Number of words in the work
*  Number of chapter in the work
*  Number of hits
*  Requiered tags
*  Additionnal tags

This code do the same as Web Srapping AO3 part 1 but only to append the database.

It was supposed to be quicker than part 1 but it's not and I don't really get why.

EDIT : So actually this code is indeed faster when you append more than 50 page, I tried to append 100 page and this code (part 2) took 368 seconds and the other (part 1) 395 seconds to run (without the graph and histogram etc).



---
# ⚠ WARNING ⚠
Two important thing to understand when you download **many files** of data :


1.   **You can't append more than 70 pages in one time**

  If you want to append a lot of data I suggest 50 page by 50 page. I mean if you want to try to append more that's your choice but the code is optimized to append 50 by 50 (or less).

  Also, one page is 20 work, so each time you run the code you can append at max 1400 works.

2.   **You need to have the SAME interval of page between files**

  Per example, you want to append the work from a ertain fandom, you have 457 pages in total. I would suggest to append page 1 to 50 (base_page_0_to_51), then 50 to 100 (base_page_50_to_101), ... , then 400 to 450 (base_page_400_to_450 and finally page 450 to 456 (base_page_450_to_457).

  For the last one you input 456 as Number_page but its append untill page 457 because my code is WEIRD.




This section doesn't have a lot of comments, if you're curious on how the code work I suggest you to check part 1 (it's the same result at the end)

**Library**

In [35]:
from bs4 import BeautifulSoup
import urllib.request
import csv
from time import sleep
import pickle
from os import path
import pandas as pd
import datetime
import random
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from google.colab import files
import pandas as pd

**Parameters**

Just as part 1, we ask you to input an URL, a first page and a last one.

In [36]:
# Choose the fandom, the beginning of the page and the ending of the page of the data you want to append

# ULR
TEST_URL = "https://archiveofourown.org/tags/Arcane:%20League%20of%20Legends%20(Cartoon%202021)/"

# First_Page
First_Page = 1
Beginning_Page = First_Page

# Last_page
Last_page = 100
Last_page = Last_page + 1
Number_page = Last_page

In [37]:
# Cleaning the tag
tag= urlparse(TEST_URL)
tag = tag.path
tag_2 = tag
tag_2 = tag_2.replace('tags','')
tag_2 = tag_2.replace('/','')
tag_2 = tag_2.replace('%20',' ')
tag_2 = tag_2.replace('*s*',' and ')

In [38]:
print("Download Data for the following tag : ",tag_2,"from page",First_Page,"to",Last_page)

Download Data for the following tag :  Arcane: League of Legends (Cartoon 2021) from page 1 to 101


**Define functions**

Download and soupify

In [39]:
def download_and_soupify(url, parser="html.parser"):
	"""
	Given a URL, downloads the site and turns it into beautiful soup.
	"""

	full_url = url + "works"
	headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
	req = urllib.request.Request(full_url, headers=headers)
	response = urllib.request.urlopen(req)
	directory_html = response.read()
	index_soup = BeautifulSoup(directory_html, parser)

	return index_soup

Get directory urls

In [40]:
def get_directory_urls(url, index_soup):
	"""
	Given the base directory as soup, figure out how many pages there are,
	and return a list of the URLs of each directory page.
	"""

	page_numbers = index_soup.find_all("ol", class_="pagination actions")

	li_entries = page_numbers[0].find_all("li")

	li_texts = [number.text for number in li_entries]
	li_digits = [int(number) for number in li_texts if number.isdigit()]
	sorted_page_numbers = sorted(li_digits)

	number_of_pages = sorted_page_numbers[-1]

	url_prefix = url + "works?page="
	directory_urls = [(url_prefix + str(page_number)) for page_number in range(0, number_of_pages+1)]

	return directory_urls

Story IDS

In [41]:
def get_story_ids(directory_url):
	"""
	Given a single directory URL, get all stories from that URL. Returns a list of story IDs.
	"""

	dir_soup = download_and_soupify(directory_url)
	dir_links = dir_soup.find_all("a", href=True)

	dir_hrefs = [link.attrs['href'] for link in dir_links]
	work_hrefs = [link for link in dir_hrefs if "/works/" in link]
	potential_ids = [work.split("/")[2] for work in work_hrefs]
	id_list = [int(work_id) for work_id in potential_ids if work_id.isdigit()]

	story_ids = list(set(id_list))

	return story_ids

Date update of the work

In [42]:
def datetime(directory_url):
  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("p",{"class":"datetime"}):
    text=x.get_text()
    date_work.append(str(text))

Title and author of the work

In [43]:
def worktitle(directory_url):

  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("h4",attrs={"class": "heading"}):
    text=x.get_text()
    work_title_authors.append(str(text))

  work_title_authors.remove('\n            Include\n          ')
  work_title_authors.remove('\n            Exclude\n          ')
  work_title_authors.remove('More Options')
  work_title_authors.remove('Pages Navigation')
  work_title_authors.remove('About the Archive')
  work_title_authors.remove('Pages Navigation')
  work_title_authors.remove('Contact Us')
  work_title_authors.remove('Development')

  (work_title_authors)

# It quit the same as the other function
# Exept this time we need to remove a lot of stuff by page to clean the text


Language of the work

In [44]:
def worklanguage(directory_url):

  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "language"}):
    text=x.get_text()
    work_language.append(str(text))

  (work_language)

# I guess you know how it works now

Number of words of the work

In [45]:
def NumberofWords(directory_url):

  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "words"}):
    text=x.get_text()
    Nbr_words.append(str(text))

Number of Chapter

In [46]:
def NumberofChapter(directory_url):

  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "chapters"}):
    text=x.get_text()
    Nbr_chapter.append(str(text))

Number of Hits

In [47]:
def NumberofHits(directory_url):

  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("dd",attrs={"class": "hits"}):
    text=x.get_text()
    Nbr_hits.append(str(text))

Requiered tags

In [48]:
def tags_req(directory_url):

  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("ul",attrs={"class": "required-tags"}):
    text=x.get_text()
    tags_requiered.append(str(text))

Additionals tags

In [49]:
def tags(directory_url):

  dir_soup = download_and_soupify(directory_url)
  for x in dir_soup.findAll("ul",attrs={"class": "tags commas"}):
    text=x.get_text()
    tags_add.append(str(text))

**Big function for everything**

In [50]:
index_soup = download_and_soupify(TEST_URL)

In [51]:
directory_urls = get_directory_urls(TEST_URL,index_soup)

In [52]:
# Empty storage
story_ids = []
story_ids_2 = []

date_work = []

work_title_authors = []

work_language = []
work_language_2 = []

Nbr_words = []

Nbr_chapter = []

Nbr_hits = []

tags_requiered = []

tags_add = []


index_page = []

In [53]:
def data_append(URL, Beginning_Page, Number_page):

  for i in range(Beginning_Page,Number_page):
    # story_ids_2
    story_ids.append(get_story_ids(directory_urls[i]))

    # data_updated
    date_work.append(datetime(directory_urls[i]))
    date_work.remove(None)

    # work_title_authors
    work_title_authors.append(worktitle(directory_urls[i]))
    work_title_authors.remove(None)

    # work_language
    work_language.append(worklanguage(directory_urls[i]))
    work_language.remove(None)

    # Number of words
    Nbr_words.append(NumberofWords(directory_urls[i]))
    Nbr_words.remove(None)

    # Number of chapter
    Nbr_chapter.append(NumberofChapter(directory_urls[i]))
    Nbr_chapter.remove(None)

    # Number of hits
    Nbr_hits.append(NumberofHits(directory_urls[i]))
    Nbr_hits.remove(None)

    # Requiered tags
    tags_requiered.append(tags_req(directory_urls[i]))
    tags_requiered.remove(None)

    # Additionnal tags
    tags_add.append(tags(directory_urls[i]))
    tags_add.remove(None)

###############################################################################

    # index_page
    for j in range(20):
      index_page.append(i)

  # work_title_authors
  for work in range(len(work_title_authors)):
    work_title_authors[work] = work_title_authors[work].replace('\n',' ')
    work_title_authors[work] = work_title_authors[work].replace('       ',' ')
    work_title_authors[work] = work_title_authors[work].replace('     ',' ')

  # work_language
  Number_page_bis = Number_page - Beginning_Page
  Number_page_bis = Number_page_bis
  Drop_language_setting = []
  for i in range(Number_page_bis):
    Drop_language_setting_bis = 20
    Drop_language_setting_bis = Drop_language_setting_bis + i*20
    Drop_language_setting.append(Drop_language_setting_bis)
  Drop_language_setting

  for i in Drop_language_setting:
    work_language.remove(work_language[i])

  # Requiered tags and additionnal tags
  for i in range(len(tags_requiered)):
    tags_requiered[i] = tags_requiered[i].replace("\n","|")
    tags_add[i] = tags_add[i].replace("\n","|")
    backslash = '"\\"'
    tags_add[i] = tags_add[i].replace(backslash,'')

**Running the function**

In [54]:
data_append(TEST_URL,First_Page,Last_page)

KeyboardInterrupt: 

In [ ]:
for i in range(0,len(story_ids)):
 story_ids_2 = story_ids_2 + story_ids[i]

# Adjust on story_ids

**Inside a DataFrame**

In [ ]:
# Dataframe
base1 = {'Index_page' : index_page,
          'ID' :story_ids_2,
          'Date_publication' : date_work,
          'Title_and_author' : work_title_authors,
          'Language_of_the_work' : work_language,
          'Word' : Nbr_words,
          'Chapter' : Nbr_chapter
          , 'Hits' : Nbr_hits
          , 'Requiered_tags' : tags_requiered
          , 'Additionnal_tags' : tags_add}
base1 = pd.DataFrame(data=base1)
base1

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

**Export the DataBase**

Download your database on your computer

In [ ]:
base_export = base1
Name_base = "base_page_" + str(Beginning_Page) + "_to_" + str(Number_page) + ".csv"
Name_base

In [ ]:
"""
from google.colab import files
base_export.to_csv(Name_base)
files.download(Name_base)
"""

Shiny ?

In [ ]:
!pip install shiny
!shiny create --template dashboard-tips

In [61]:
!pip install -r dashboard-tipsdoryane/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.2/607.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.6 MB/s eta 0:00:00


In [ ]:
!shiny run dashboard-tipsdoryane/app.py

In [ ]:
!pip install shiny pyngrok

In [ ]:
!pip install shiny pyngrok
!pip install flask_ngrok

In [6]:
!ngrok authtoken 2sE7HpgvNmzZ3nGRtpZAjOxPc8j_8a86AU39jtr2VX4j7uTKS

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
!cd dashboard-tipsdoryane

In [8]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

@app.route('/')
def index():
  # Replace with your Shiny app's index.html
  return render_template('index.html')

if __name__ == '__main__':
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           